In [0]:
%tensorflow_version 1.x

import numpy as np
import keras
from keras.datasets import mnist
import sys
from scipy.stats import entropy
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, SpatialDropout2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.regularizers import l2
from keras import backend as K

from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
def predict_with_uncertainty(f, x, n_iter=100):
    """Function generating non-deterministic predictions using MC dropout and returning the mean of these predictions
    Adapted from: https://stackoverflow.com/questions/43529931/how-to-calculate-prediction-uncertainty-using-keras
    #Arguments:
        f: function mapping model input and Keras backend learning_phase flag to model output
        x: input
        n_iter: number of repreated MC dropout predictions per point
    #Returns:
        Mean of MC dropout predictions
    """
    result = np.zeros((n_iter,x.shape[0], 2))
    for i in range(n_iter):
        predictions = np.array(f((x, 1))[0])
        result[i,:, :] = predictions
    prediction = result.mean(axis=0)
    return prediction

#Used for making repeated pool predictions
def predict_pool_with_uncertainty(f, x, n_iter=50):
    """Function generating and returning non-deterministic predictions using MC dropout
    Adapted from: https://stackoverflow.com/questions/43529931/how-to-calculate-prediction-uncertainty-using-keras
    #Arguments:
        f: function mapping model input and Keras backend learning_phase flag to model output
        x: input
        n_iter: number of repreated MC dropout predictions per point
    #Returns:
        All MC dropout predictions
    """
    result = np.zeros((n_iter,x.shape[0], 2))
    for i in range(n_iter):
        predictions = np.array(f((x, 1))[0])
        result[i,:, :] = predictions
    return result


def run_model (train_data_indices, pool_sample_indices):
    """Initializes and trains a Keras model from scratch on the given training data, returning test MAE and MSE (and MC dropout predictions for D_{sample})
    #Arguments
        train_data_indices: indices within X_train_All of current training points
        pool_sample_indices: indices within X_train_All of pool points in the selected sample D_{sample}
    #Returns
        Test accuracy and MC dropout predictions for all pool points in D_{sample}
    """
  X_train = np.expand_dims(X_train_All[train_data_indices], axis=1)
  y_train = y_train_All[train_data_indices]
  y_train[y_train==7] = 1
  y_train[y_train==9] = 0
  y_train = keras.utils.to_categorical(y_train, num_classes=2)
  train_size = y_train.shape[0]
  Weight_Decay = 0.0005/train_size
  dropout_prob = 0.30
  batch_size=128
  nb_filters = 35
  nb_pool = 3
  nb_conv = 4
  img_rows = img_cols = 28
  nb_classes = 2
  model = Sequential()
  model.add(Convolution2D(nb_filters, nb_conv, strides=1, data_format="channels_first", input_shape=(1, img_rows, img_cols)))
  model.add(Activation('relu'))
  model.add(Convolution2D(nb_filters, nb_conv, data_format="channels_first", strides=2))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool), data_format="channels_first"))
  model.add(Dropout(dropout_prob))
  model.add(Flatten())
  model.add(Dense(128, W_regularizer=l2(Weight_Decay)))
  model.add(Activation('relu'))
  model.add(Dropout(dropout_prob))
  model.add(Dense(nb_classes, W_regularizer=l2(Weight_Decay)))
  model.add(Activation('softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam')
  model.fit(X_train, y_train, epochs=300, batch_size=128, verbose=0)
  f = K.function([model.layers[0].input, K.learning_phase()],[model.layers[-1].output])
  y_test_output = predict_with_uncertainty(f, X_test, n_iter=100)
  y_test_predictions = np.argmax(y_test_output, axis=1)
  pool_sample_predictions = predict_pool_with_uncertainty(f, np.expand_dims(X_train_All[pool_sample_indices], axis=1), n_iter=50)
  return [np.sum(y_test_predictions==y_test_original)/(y_test_original.shape[0]), pool_sample_predictions]



def get_acquisition_fn(pool_forward_pass):
  """Evaluates Max Entropy acquisition function (pool_forward_pass shape is [num_masks (J), number of pool candidate points, number of classes])
  #Arguments:
      pool_forward_pass: MC dropout predictions on candidate points for acquisition in the pool
  #Returns:
      Value of Max Entropy acquisition function for each pool candidate point
  """
  average_predictions = np.mean(pool_forward_pass, axis=0)
  all_entropies = entropy(average_predictions, axis=1) 
  return all_entropies

In [0]:
#Active learning Parameters/Settings
dropout_prob = 0.3
num_experiments = 3
num_acquisitions = 1000
pool_sample_size = 2500
num_masks = 50

batch_size = 1


In [0]:
#Loading Data
data_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/MNIST/"
starting_ind_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/MNIST/Binary_7_9_AL_Scripts/"
results_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/MNIST/Binary_7_9_AL_Results/"
train_data = np.loadtxt(data_path + "mnist_train.csv", 
			delimiter=",")
test_data = np.loadtxt(data_path + "mnist_test.csv", 
			delimiter=",") 
y_train_All = train_data[:,0]
y_test = test_data[:,0]
X_train_All = train_data[:,1:].reshape((60000,28,28))
X_test = test_data[:,1:].reshape((10000,28,28)) 
train_ind = np.concatenate((np.argwhere(y_train_All==7), np.argwhere(y_train_All==9))).flatten()
test_ind = np.concatenate((np.argwhere(y_test==7), np.argwhere(y_test==9))).flatten()
y_test = y_test[test_ind]
y_test[y_test==9] = 0
y_test[y_test==7]= 1
y_test_original = y_test
y_test = keras.utils.to_categorical(y_test, num_classes=2)
X_test = np.expand_dims(X_test[test_ind], axis=1)

#Iterating across experiments, each of which begins with a different training set (that is balanced across classes)
#Note: code is currently set up to *resume* an active learning experiment
#To start an active learning experiment from scratch, several lines below that are commented out should be uncommented, and vice versa

for e in range(0,1):
  acc_file = "MaxEnt_BS"+str(batch_size)+"_Acc_Ind" + str(e+1) + ".npy"
  ind_file = "MaxEnt_BS"+str(batch_size)+"_Ind_Ind" + str(e+1) + ".npy"
  #exp_acc = []
  exp_acc = list(np.load(results_path+acc_file))
  #train_data_indices = list(np.load(starting_ind_path + 'trainindices' + str(e+1) + '.npy'))
  train_data_indices = list(np.load(results_path+ind_file))
  num_acquisitions = num_acquisitions - batch_size * (len(exp_acc) - 1)
  pool_indices = [i for i in train_ind if i not in train_data_indices]
  #Looping over active learning iterations
  for acq in range(num_acquisitions//batch_size + 1):
    #Selecting candidate points for acquisition
    pool_ind_sample = np.random.choice(pool_indices, pool_sample_size, replace=False)
    #Obtaining current model performance and MC dropout pool predictions
    model_results = run_model(train_data_indices, pool_ind_sample)
    if acq != 0:
      exp_acc.append(model_results[0])
    pool_forward_pass_results = model_results[1]  
    #Maximum entropy acquisition of new points
    acq_fn_values = get_acquisition_fn(pool_forward_pass_results) 
    all_acq_ind_ind = np.flip(np.argsort(acq_fn_values))[0:batch_size]
    for acq_ind_ind in all_acq_ind_ind:
      train_data_indices.append(pool_ind_sample[acq_ind_ind])
      pool_indices.remove(pool_ind_sample[acq_ind_ind])
    #Saving results
    np.save(results_path+acc_file, np.array(exp_acc))
    np.save(results_path+ind_file, np.array(train_data_indices))
    print('Exp ' + str(e+1) + ', Number elapsed iterations: ' + str(len(exp_acc)) + ", last acc: " + str(exp_acc[-1]))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, kernel_regularizer=<keras.reg...)`


Exp 1, Number elapsed iterations: 1001, last acc: 0.9945999018163967
